<a href="https://colab.research.google.com/github/hoaint25/Applied_charting_plotting_in_python/blob/main/Week2_Assignment/W2_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment 2
Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to Preview the Grading for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) Daily Global Historical Climatology Network (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

id : station identification code
date : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
element : indicator of element type
TMAX : Maximum temperature (tenths of degrees C)
TMIN : Minimum temperature (tenths of degrees C)
value : data value for element (tenths of degrees C)
For this assignment, you must:

Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.
The data you have been given is near Ann Arbor, Michigan, United States, and the stations the data comes from are shown on the map 

##Get the data from the source 

In [ ]:
binsize = 400 
hashid = 'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89'
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d{}/{}.csv'.format(binsize,hashid))
df['Data_Value'] = df['Data_Value'].apply(lambda x: x/10)
#print(df['Data_Value'])
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
#print(df)

#remove leap year 
s_o = df.shape #return the number rows and columns
#print(s_o)
df = df[~((df['Month'] ==2) & (df['Day'] == 29))]
s_r = df.shape
print('remove {} series of data'.format(s_o[0]-s_r[0]))

#max and min data after grouping by each day 
max_05_14 = df[(df['Element'] == 'TMAX') & (df['Year'] >=2005) & (df['Year'] < 2015)].groupby(['Month','Day']).aggregate({'Data_Value':np.max})
min_05_14 = df[(df['Element'] == 'TMIN') & (df['Year'] >=2005) & (df['Year'] < 2015)].groupby(['Month','Day']).aggregate({'Data_Value':np.min})

max_15 = df[(df['Element'] == 'TMAX') & (df['Year'] == 2015)].groupby(['Month','Day']).aggregate({'Data_Value':np.max})
min_15 = df[(df['Element'] == 'TMIN') & (df['Year'] == 2015)].groupby(['Month','Day']).aggregate({'Data_Value':np.min})

broken_max = np.where(max_15['Data_Value'] > max_05_14['Data_Value'])[0]
broken_min = np.where(min_15['Data_Value'] < min_05_14['Data_Value'])[0]


##Plot the graph

In [ ]:
import numpy as np
plt.figure()
plt.plot(max_05_14.values, label = 'Max Temp (2004-2015)',linewidth=1, alpha = 0.7, c='r')
plt.plot(min_05_14.values, label = 'Min Temp (2004-2015)',linewidth=1, alpha = 0.7, c='b')
plt.gca().fill_between(range(len(max_05_14)), max_05_14['Data_Value'], min_05_14['Data_Value'], facecolor='yellow', alpha=0.25)

plt.scatter(broken_max, max_15.iloc[broken_max], s=10, c='red', label='High Temp Broken (2015)')
plt.scatter(broken_min, min_15.iloc[broken_min], s=10, c='blueviolet', label='Low Temp Broken (2015)')
#add more in chart
plt.legend(loc = 'best', title = 'Temperature', fontsize = 8)

# tick label in chart 
plt.xticks(np.linspace(0,30 + 30*11, num= 12),( r'Jan', r'Feb', r'Mar', r'Apr', r'May', r'Jun', r'Jul', r'Aug', r'Sep', r'Oct', r'Nov', r'Dec'), alpha=0.8)
plt.yticks(alpha = 0.8)

#xlabel and ylabel
plt.xlabel('Months',alpha = 0.8)
plt.ylabel('Temperatute',alpha = 0.8)
plt.title('Temperature Plot', alpha = 0.8)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['bottom'].set_alpha(0.3)
plt.gca().spines['left'].set_alpha(0.3)
